<a href="https://colab.research.google.com/github/henryantwi/-Profile/blob/main/Learn_How_Vector_Stores_Work_by_Building_a_Recommendation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learn How Vector Stores Work by Building a Recommendation Engine

This tutorial provides a fundamental demonstration of how vector databases operate.

In this example, we will define various parameters, or dimensions, for food critiquing. The engine will then suggest dishes to the user based on these parameters, aligned with their preferences. For vector storage and management, we will utilize Upstash Vector.

Outline:

1- Create an index on Upstash Vector and install the required
dependencies.

2- Define dimensions for food critiqueing.

3- Insert data.

4- Create query by asking preferences to user.

5- Show recommended dishes by querying the vector database.

Let's start!



```
# This is formatted as code
```

## Create Index
Head out to [Upstash Console](https://console.upstash.com) to create an Index with 24 dimensions under Vector tab.

Copy the `UPSTASH_VECTOR_REST_URL` and `UPSTASH_VECTOR_REST_TOKEN` values for the next step.

In [ ]:
UPSTASH_VECTOR_REST_URL="URL"
UPSTASH_VECTOR_REST_TOKEN="TOKEN"

## Install `upstash_vector` Python SDK and initialize your client.

In [ ]:
%pip install upstash_vector -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


Paste UPSTASH_VECTOR_REST_URL and UPSTASH_VECTOR_REST_TOKEN below.

In [ ]:
from upstash_vector import Index

index = Index(UPSTASH_VECTOR_REST_URL, UPSTASH_VECTOR_REST_TOKEN)
index.reset() # Remove all vectors in the index
index.info()

# Define dimensions for food critiqueing

Now, instead of using an embedding an already existing text, image or a sound sample with a predefined dimension count, we will define our own dimensions for our vectors.

Here, we can define our dimensions for critiqueing the food. In this example, following 24 dimensions are defined:


In [ ]:
food_dimensions = ["sweetness", "saltiness", "sourness", "bitterness", "umami" ,"crispiness" , "tenderness", "creaminess", "chewiness", "fragrance", "intensity", "visual_appeal", "plating", "temperature", "creaminess_mouthfeel", "spiciness_mouthfeel", "astringency_mouthfeel", "layering_complexity", "contrast_complexity", "quality_of_ingredients_freshness", "cooking_method_technique", "seasoning_technique", "creativity_originality", "innovation_originality"]
print(f"Following {len(food_dimensions)} dimensions are defined for rating food in different aspects: {food_dimensions}")

Following 24 dimensions are defined for rating food in different aspects: ['sweetness', 'saltiness', 'sourness', 'bitterness', 'umami', 'crispiness', 'tenderness', 'creaminess', 'chewiness', 'fragrance', 'intensity', 'visual_appeal', 'plating', 'temperature', 'creaminess_mouthfeel', 'spiciness_mouthfeel', 'astringency_mouthfeel', 'layering_complexity', 'contrast_complexity', 'quality_of_ingredients_freshness', 'cooking_method_technique', 'seasoning_technique', 'creativity_originality', 'innovation_originality']


## Define some helper functions for normalizing

We want to normalize our vector values since sum of all those dimensions might not add upto the same value for different dishes in our existing database. This way, we can keep the ratio between different aspects of the dish.

In [ ]:
def highest_element(vector):
  highest_score = 0
  for element in vector:
    if element > highest_score:
      highest_score = element

  return highest_score

def vector_to_normalized_vector(vector):
  division_factor = highest_element(vector)
  return [x/division_factor for x in vector]


def dish_to_normalized_vector(id, name, origin, cuisine, description,
                 sweetness, saltiness, sourness, bitterness, umami,
                 crispiness, tenderness, creaminess, chewiness,
                 fragrance, intensity,
                 visual_appeal, plating,
                 temperature,
                 creaminess_mouthfeel, spiciness_mouthfeel, astringency_mouthfeel,
                 layering_complexity, contrast_complexity,
                 quality_of_ingredients_freshness,
                 cooking_method_technique, seasoning_technique,
                 creativity_originality, innovation_originality):


  metadata = {"origin": origin, "cuisine": cuisine, "description": description, "name": name}
  vector = [sweetness, saltiness, sourness, bitterness, umami,
                 crispiness, tenderness, creaminess, chewiness,
                 fragrance, intensity,
                 visual_appeal, plating,
                 temperature,
                 creaminess_mouthfeel, spiciness_mouthfeel, astringency_mouthfeel,
                 layering_complexity, contrast_complexity,
                 quality_of_ingredients_freshness,
                 cooking_method_technique, seasoning_technique,
                 creativity_originality, innovation_originality
            ]
  vector = vector_to_normalized_vector(vector)

  return (id, vector, metadata)

## Example Insertions
We will insert 2 example dishes to solidify the use case.

As you can see, the sum of the parameters do not add up to th same value, however by normalizing them, we keep our database available for queries without worrying about the exact values for ratings.

In [ ]:
dish1_vector = dish_to_normalized_vector(
    "custom-dish-0",
    name="Grilled Salmon with Lemon Herb Butter Sauce",
    origin="North America",
    cuisine="American",
    description="A classic dish featuring grilled salmon with a flavorful lemon herb butter sauce.",
    sweetness=5, saltiness=10, sourness=6, bitterness=3, umami=12,
    crispiness=8, tenderness=10, creaminess=6, chewiness=2,
    fragrance=10, intensity=8,
    visual_appeal=12, plating=10,
    temperature=12,
    creaminess_mouthfeel=6, spiciness_mouthfeel=2, astringency_mouthfeel=2,
    layering_complexity=8, contrast_complexity=9,
    quality_of_ingredients_freshness=10,
    cooking_method_technique=12, seasoning_technique=10,
    creativity_originality=8, innovation_originality=7
)

dish2_vector = dish_to_normalized_vector(
    "custom-dish-1",
    name="Vegetarian Stir-Fried Tofu with Mixed Vegetables",
    origin="Asia",
    cuisine="Asian",
    description="A delightful vegetarian stir-fry featuring tofu and a medley of mixed vegetables.",
    sweetness=7, saltiness=8, sourness=9, bitterness=4, umami=10,
    crispiness=8, tenderness=10, creaminess=8, chewiness=6,
    fragrance=8, intensity=7,
    visual_appeal=8, plating=7,
    temperature=9,
    creaminess_mouthfeel=8, spiciness_mouthfeel=5, astringency_mouthfeel=4,
    layering_complexity=6, contrast_complexity=7,
    quality_of_ingredients_freshness=8,
    cooking_method_technique=8, seasoning_technique=8,
    creativity_originality=8, innovation_originality=7
)

# Printing the vectors
print("Custom Dish 1 Normalized Vector:")
print(dish1_vector)
print()
print("Custom Dish 2 Normalized Vector:")
print(dish2_vector)


Custom Dish 1 Normalized Vector:
('custom-dish-0', [0.4166666666666667, 0.8333333333333334, 0.5, 0.25, 1.0, 0.6666666666666666, 0.8333333333333334, 0.5, 0.16666666666666666, 0.8333333333333334, 0.6666666666666666, 1.0, 0.8333333333333334, 1.0, 0.5, 0.16666666666666666, 0.16666666666666666, 0.6666666666666666, 0.75, 0.8333333333333334, 1.0, 0.8333333333333334, 0.6666666666666666, 0.5833333333333334], {'origin': 'North America', 'cuisine': 'American', 'description': 'A classic dish featuring grilled salmon with a flavorful lemon herb butter sauce.', 'name': 'Grilled Salmon with Lemon Herb Butter Sauce'})

Custom Dish 2 Normalized Vector:
('custom-dish-1', [0.7, 0.8, 0.9, 0.4, 1.0, 0.8, 1.0, 0.8, 0.6, 0.8, 0.7, 0.8, 0.7, 0.9, 0.8, 0.5, 0.4, 0.6, 0.7, 0.8, 0.8, 0.8, 0.8, 0.7], {'origin': 'Asia', 'cuisine': 'Asian', 'description': 'A delightful vegetarian stir-fry featuring tofu and a medley of mixed vegetables.', 'name': 'Vegetarian Stir-Fried Tofu with Mixed Vegetables'})


## Upsert and Fetch

In [ ]:
# Upsert the food vectors
index.upsert([dish1_vector, dish2_vector])

'Success'

In [ ]:
index.fetch("custom-dish-1", include_metadata=True)

[FetchResult(id='custom-dish-1', vector=None, metadata={'origin': 'Asia', 'cuisine': 'Vegetarian', 'description': 'A delightful vegetarian stir-fry featuring tofu and a medley of mixed vegetables.', 'name': 'Vegetarian Stir-Fried Tofu with Mixed Vegetables'})]

## Asking Customer for the Preferences and Printing

In [ ]:
def get_customer_preferences():

  customer_preferences = []

  limit = 120
  print(f"We will ask you to rate the importance of the following criteria for the food you want to select: {food_dimensions}")
  print(f"Your total points cannot exceed {limit}")
  for dimension in food_dimensions:
    dimension_value = input(f"(Remaining points {limit}) {dimension}: ")

    try:
      dimension_value = float(dimension_value)
    except Exception:
      print("Value was not successfully parsed. Using 0 instead.")
      customer_preferences.append(0)
      continue

    if limit < dimension_value or dimension_value < 0:
      print("value cannot be higher than remaining limit or negative. Using 0 instead.")
      customer_preferences.append(0)
      continue

    customer_preferences.append(dimension_value)
    limit -= dimension_value

  return customer_preferences

def print_recommended_dishes(dishes):
  print("Here are the recommended dishes for your taste:\n")
  for dish in suggested_dishes:
    print(f"Match: {dish.metadata['name']}")
    print(f"Match Score: {dish.score}")
    print(f"Origin: {dish.metadata['origin']}")
    print(f"Cuisine: {dish.metadata['cuisine']}")
    print(f"Description: {dish.metadata['description']}")
    print()



## Get Recommended Dishes for Customer Preferences

In [ ]:
customer_preferences = get_customer_preferences()

We will ask you to rate the importance of the following criteria for the food you want to select: ['sweetness', 'saltiness', 'sourness', 'bitterness', 'umami', 'crispiness', 'tenderness', 'creaminess', 'chewiness', 'fragrance', 'intensity', 'visual_appeal', 'plating', 'temperature', 'creaminess_mouthfeel', 'spiciness_mouthfeel', 'astringency_mouthfeel', 'layering_complexity', 'contrast_complexity', 'quality_of_ingredients_freshness', 'cooking_method_technique', 'seasoning_technique', 'creativity_originality', 'innovation_originality']
Your total points cannot exceed 120
(Remaining points 120) sweetness: 10
(Remaining points 110.0) saltiness: 20
(Remaining points 90.0) sourness: 10
(Remaining points 80.0) bitterness: 10
(Remaining points 70.0) umami: 5
(Remaining points 65.0) crispiness: 5
(Remaining points 60.0) tenderness: 10
(Remaining points 50.0) creaminess: 1
(Remaining points 49.0) chewiness: 1
(Remaining points 48.0) fragrance: 10
(Remaining points 38.0) intensity: 10
(Remaining

#### Normalize Customer Preferences for Ease of Querying

In [ ]:
customer_preferences_normalized = vector_to_normalized_vector(customer_preferences)

### Define Parameter for Querying



In [ ]:
top_k = 1 # How many recommendations there will be

In [ ]:
suggested_dishes = index.query(customer_preferences_normalized, top_k=top_k, include_metadata=True)

In [ ]:
print_recommended_dishes(suggested_dishes)

Here are the recommended dishes for your taste:

Match: Vegetarian Stir-Fried Tofu with Mixed Vegetables
Match Score: 0.8477832
Origin: Asia
Cuisine: Asian
Description: A delightful vegetarian stir-fry featuring tofu and a medley of mixed vegetables.



## Define More Dishes

Here, we can generate some more dishes so that our query pool will be larger.

For the vector values, we will auto-generate some float numbers. In real life scenarios, those values may come from customer reviews or from chefs who cook the dishes.

In [ ]:
import random

class Dish:
    def __init__(self, name, cuisine, origin, description):
        self.name = name
        self.cuisine = cuisine
        self.origin = origin
        self.description = description
        self.vector = [random.random() for _ in range(24)] # food dimension is 24

dishes = [
    Dish(name="Spaghetti Bolognese", cuisine="Italian", origin="Italy", description="Classic Italian pasta dish with a rich meat sauce."),
    Dish(name="Chicken Alfredo Pasta", cuisine="Italian", origin="Italy", description="Creamy Alfredo pasta with grilled chicken, perfect for pasta lovers."),
    Dish(name="Margherita Pizza", cuisine="Italian", origin="Italy", description="A timeless pizza topped with fresh tomatoes, mozzarella, and basil."),
    Dish(name="Beef Tacos", cuisine="Mexican", origin="Mexico", description="Classic beef tacos with seasoned meat, fresh toppings, and salsa."),
    Dish(name="Caesar Salad", cuisine="International", origin="Mexico", description="A classic salad with crisp romaine, parmesan, croutons, and Caesar dressing."),
    Dish(name="Shrimp Scampi", cuisine="Italian", origin="Italy", description="Garlic butter shrimp served over pasta or rice."),
    Dish(name="Grilled Salmon", cuisine="Seafood", origin="International", description="Perfectly grilled salmon seasoned with herbs and served with lemon."),
    Dish(name="Pad Thai", cuisine="Thai", origin="Thailand", description="A flavorful Thai stir-fried noodle dish with shrimp, peanuts, and lime."),
    Dish(name="Caprese Salad", cuisine="Italian", origin="Italy", description="A refreshing salad with tomatoes, fresh mozzarella, basil, and balsamic glaze."),
    Dish(name="Sushi Rolls", cuisine="Japanese", origin="Japan", description="Assorted sushi rolls with fresh fish, rice, and seaweed."),
    Dish(name="Chicken Parmesan", cuisine="Italian", origin="Italy", description="Breaded and fried chicken topped with marinara sauce and melted cheese."),
    Dish(name="BBQ Ribs", cuisine="American", origin="United States", description="Slow-cooked ribs glazed with barbecue sauce for a smoky flavor."),
    Dish(name="Vegetable Stir-Fry", cuisine="Asian", origin="Various", description="Colorful mix of vegetables stir-fried in a savory sauce."),
    Dish(name="Lobster Bisque", cuisine="French", origin="France", description="Creamy soup with rich lobster flavor, often served as a luxurious appetizer."),
    Dish(name="Fettuccine Alfredo", cuisine="Italian", origin="Italy", description="Fettuccine pasta in a creamy Alfredo sauce made with butter and parmesan."),
    Dish(name="Tandoori Chicken", cuisine="Indian", origin="India", description="Chicken marinated in yogurt and spices, cooked in a tandoor for a smoky flavor."),
    Dish(name="Chicken Noodle Soup", cuisine="American", origin="United States", description="Comforting soup with chicken, noodles, and vegetables."),
    Dish(name="Greek Salad", cuisine="Greek", origin="Greece", description="Fresh salad with tomatoes, cucumbers, olives, feta cheese, and Greek dressing."),
    Dish(name="Fish and Chips", cuisine="British", origin="United Kingdom", description="Fried fish fillets served with crispy fries, a popular pub dish."),
    Dish(name="Pulled Pork Sandwich", cuisine="American", origin="United States", description="Slow-cooked pulled pork piled on a bun, often topped with coleslaw."),
    Dish(name="Mushroom Risotto", cuisine="Italian", origin="Italy", description="Creamy risotto cooked with mushrooms, onions, and Arborio rice."),
    Dish(name="Chicken Teriyaki", cuisine="Japanese", origin="Japan", description="Grilled chicken glazed with a sweet and savory teriyaki sauce."),
    Dish(name="Eggplant Parmesan", cuisine="Italian", origin="Italy", description="Breaded and baked eggplant slices topped with marinara sauce and melted cheese."),
    Dish(name="Clam Chowder", cuisine="American", origin="United States", description="Creamy soup with clams, potatoes, onions, and bacon."),
    Dish(name="Chicken Caesar Wrap", cuisine="American", origin="United States", description="Caesar salad with grilled chicken wrapped in a tortilla."),
    Dish(name="Pesto Pasta", cuisine="Italian", origin="Italy", description="Pasta tossed in a vibrant pesto sauce made with basil, garlic, pine nuts, and Parmesan."),
    Dish(name="Beef Stroganoff", cuisine="Russian", origin="Russia", description="Sliced beef in a creamy mushroom sauce, typically served over egg noodles."),
    Dish(name="Lobster Roll", cuisine="American", origin="United States", description="Butter-poached lobster meat served in a split-top bun, often with mayo and lemon."),
    Dish(name="Chicken Shawarma", cuisine="Middle Eastern", origin="Lebanon", description="Marinated and grilled slices of chicken, often served in a flatbread with veggies and tahini."),
    Dish(name="Gyoza (Dumplings)", cuisine="Japanese", origin="Japan", description="Japanese dumplings filled with meat and vegetables, usually pan-fried or steamed."),
    Dish(name="Chicken Quesadilla", cuisine="Mexican", origin="Mexico", description="Grilled tortilla filled with melted cheese and seasoned chicken."),
    Dish(name="Ratatouille", cuisine="French", origin="France", description="Vegetable stew made with eggplant, zucchini, bell peppers, tomatoes, and herbs."),
    Dish(name="Falafel Wrap", cuisine="Middle Eastern", origin="Middle East", description="Deep-fried chickpea patties served in a wrap with veggies and tahini sauce."),
    Dish(name="Beef Bulgogi", cuisine="Korean", origin="South Korea", description="Marinated and grilled slices of beef, often served with rice and vegetables."),
    Dish(name="Margherita Flatbread", cuisine="Italian", origin="Italy", description="Flatbread topped with fresh tomatoes, mozzarella, and basil."),
    Dish(name="Tom Yum Soup", cuisine="Thai", origin="Thailand", description="Spicy and sour Thai soup with shrimp, mushrooms, and lemongrass."),
    Dish(name="Ceviche", cuisine="Peruvian", origin="Peru", description="Citrus-marinated raw fish or seafood, often served with onions, peppers, and cilantro."),
    Dish(name="Chicken Marsala", cuisine="Italian", origin="Italy", description="Chicken cooked in a Marsala wine sauce with mushrooms."),
    Dish(name="Philly Cheesesteak", cuisine="American", origin="United States", description="Sliced beefsteak with melted cheese, often served in a roll."),
    Dish(name="Quinoa Salad", cuisine="International", origin="Various", description="Salad made with quinoa, vegetables, and a flavorful dressing."),
    Dish(name="Baked Ziti", cuisine="Italian", origin="Italy", description="Baked pasta dish with ziti, marinara sauce, and melted cheese."),
    Dish(name="Lobster Mac and Cheese", cuisine="American", origin="United States", description="Macaroni and cheese with chunks of lobster, a decadent comfort food."),
    Dish(name="Miso Soup", cuisine="Japanese", origin="Japan", description="Japanese soup with soybean paste, tofu, seaweed, and green onions."),
    Dish(name="Chicken Fajitas", cuisine="Mexican", origin="Mexico", description="Sizzling grilled chicken with onions and bell peppers, served with tortillas."),
    Dish(name="Vegetable Lasagna", cuisine="Italian", origin="Italy", description="Lasagna made with layers of pasta, ricotta, marinara sauce, and assorted vegetables."),
    Dish(name="Tom Kha Gai", cuisine="Thai", origin="Thailand", description="Thai coconut soup with chicken, mushrooms, and galangal."),
    Dish(name="Chicken Piccata", cuisine="Italian", origin="Italy", description="Chicken sautéed in a lemon and caper sauce."),
    Dish(name="Peking Duck", cuisine="Chinese", origin="China", description="Crispy duck skin served with thin pancakes, hoisin sauce, and sliced scallions."),
    Dish(name="Lobster Thermidor", cuisine="French", origin="France", description="Lobster meat in a creamy brandy-infused sauce, typically served in the lobster shell."),
    Dish(name="Chicken Korma", cuisine="Indian", origin="India", description="Chicken cooked in a rich and creamy cashew and almond sauce."),
    Dish(name="Szechuan Tofu", cuisine="Chinese", origin="China", description="Spicy tofu dish with Szechuan peppercorns and a flavorful sauce."),
    Dish(name="Hawaiian Poke Bowl", cuisine="Hawaiian", origin="Hawaii", description="Raw fish salad served in a bowl with rice, vegetables, and flavorful sauces."),
    Dish(name="Buffalo Chicken Wings", cuisine="American", origin="United States", description="Chicken wings coated in buffalo sauce, often served with celery and blue cheese dressing."),
    Dish(name="Mushroom Swiss Burger", cuisine="American", origin="United States", description="Burger topped with sautéed mushrooms and melted Swiss cheese."),
    Dish(name="Chicken Gyro", cuisine="Greek", origin="Greece", description="Grilled chicken wrapped in a flatbread with tomatoes, onions, and tzatziki sauce."),
    Dish(name="French Onion Soup", cuisine="French", origin="France", description="Soup made with caramelized onions, beef broth, and topped with melted cheese."),
    Dish(name="Beef Wellington", cuisine="British", origin="United Kingdom", description="Beef fillet wrapped in puff pastry and baked to perfection."),
]

## Batch Upsert the Dish Vectors

In [ ]:
entries = []
for i in range(len(dishes)):
  dish = dishes[i]
  vector_entry = {"id": f"dish-{i}", "vector": dish.vector, "metadata": {"name": dish.name, "cuisine": dish.cuisine, "origin": dish.origin, "description": dish.description}}
  entries.append(vector_entry)

index.upsert(entries)

'Success'

## Query for User Preferences

Here, we can increase the recommended dish count, since we have more data.

In [ ]:
top_k = 10

In [ ]:
customer_preferences = get_customer_preferences()
customer_preferences_normalized = vector_to_normalized_vector(customer_preferences)

We will ask you to rate the importance of the following criteria for the food you want to select: ['sweetness', 'saltiness', 'sourness', 'bitterness', 'umami', 'crispiness', 'tenderness', 'creaminess', 'chewiness', 'fragrance', 'intensity', 'visual_appeal', 'plating', 'temperature', 'creaminess_mouthfeel', 'spiciness_mouthfeel', 'astringency_mouthfeel', 'layering_complexity', 'contrast_complexity', 'quality_of_ingredients_freshness', 'cooking_method_technique', 'seasoning_technique', 'creativity_originality', 'innovation_originality']
Your total points cannot exceed 120
(Remaining points 120) sweetness: 10
(Remaining points 110.0) saltiness: 10
(Remaining points 100.0) sourness: 20
(Remaining points 80.0) bitterness: 10
(Remaining points 70.0) umami: 10
(Remaining points 60.0) crispiness: 40
(Remaining points 20.0) tenderness: 20
(Remaining points 0.0) creaminess: 
Value was not successfully parsed. Using 0 instead.
(Remaining points 0.0) chewiness: 
Value was not successfully parsed. 

In [ ]:
suggested_dishes = index.query(customer_preferences_normalized, top_k=top_k, include_metadata=True)

print_recommended_dishes(suggested_dishes)

Here are the recommended dishes for your taste:

Match: Vegetarian Stir-Fried Tofu with Mixed Vegetables
Match Score: 2.4499998
Origin: Asia
Cuisine: Vegetarian
Description: A delightful vegetarian stir-fry featuring tofu and a medley of mixed vegetables.

Match: Spaghetti Bolognese
Match Score: 2.3085508
Origin: Italy
Cuisine: Italian
Description: Classic Italian pasta dish with a rich meat sauce.

Match: Hawaiian Poke Bowl
Match Score: 2.291547
Origin: Hawaii
Cuisine: Hawaiian
Description: Raw fish salad served in a bowl with rice, vegetables, and flavorful sauces.

Match: Buffalo Chicken Wings
Match Score: 2.1193745
Origin: United States
Cuisine: American
Description: Chicken wings coated in buffalo sauce, often served with celery and blue cheese dressing.

Match: Grilled Salmon with Lemon Herb Butter Sauce
Match Score: 2.0923915
Origin: North America
Cuisine: Seafood
Description: A classic dish featuring grilled salmon with a flavorful lemon herb butter sauce.

Match: Chicken Caesa

## Outro

In this example, we demonstrated the use of a vector database in recommendation systems, showing that it can effectively suit your use case when paired with a reliable dataset.

Beyond this Dish Recommendation system, vector databases can be applied in numerous other fields. In these fields, objects may be represented by a wide range of dimensions, potentially numbering in the hundreds or thousands, tailored to the specific use case.

While in this example we used our own definitions and dimension count, there are many embedding services, such as OpenAI, offering predefined dimension sizes for their embedding services.

Checkout [our examples](https://drive.google.com/drive/folders/1_W7MgkKGJmbfVQ_QiW_6qcfq0JZYFnhw?usp=sharing) for more ai notebooks and [follow us on X](https://twitter.com/upstash) for product updates.